In [1]:
from itertools import product, permutations
from subprocess import Popen

In [2]:
from random import randint, shuffle

In [3]:
models = [
#         "aloxatel/bert-base-mnli", 
#           "textattack/bert-base-uncased-CoLA", 
#           "textattack/bert-base-uncased-RTE", 
# #           "aloxatel/mbert", 
          "bert-base-uncased",
#           "deepset/bert-base-cased-squad2",
#           "textattack/roberta-base-MNLI",
         ]
k_vals = [1,3,5]
logging_steps_vals = [50]
epochs_vals = [3]
warmup_steps = [50]
grad_acc_steps_vals = [16]
lrs = [#1e-4, 
       5e-5]
ld = ['', '--levenshtein_only']
# ctxs = [
# #     "", 
#         "--include_context"
#        ]

In [4]:
prod = product(
    models,
    k_vals, 
    logging_steps_vals,
    epochs_vals,
    warmup_steps,
    grad_acc_steps_vals,
    lrs,
    ld#ctxs
)
prod = [*prod]
devices = [*permutations('0123', 3)]
shuffle(devices)

cmds = []
for i, (model, k, logging_steps, epochs, warmup, grad_acc_steps, lr, ctx) in enumerate(prod):
    dev = devices[i%len(devices)]
    cmd = f'''\
    conda activate wfc; \
    cd ~/wfc-hons; \
    CUDA_VISIBLE_DEVICES={','.join(dev)} \
    ./run_wfc.py --model_name_or_path {model} --do_train --do_eval --output_dir out_fever_dbg \
    --top_k {k} --logging_steps {logging_steps} --num_train_epochs {epochs} --warmup_steps {warmup} --gradient_accumulation_steps {grad_acc_steps} \
    --learning_rate {lr} {ctx} --overwrite_output_dir --maxnum {30_000} \
          '''
#     print(cmd)
    cmds += [cmd]
    
print(*cmds, sep='\n\n')

    conda activate wfc;     cd ~/wfc-hons;     CUDA_VISIBLE_DEVICES=1,0,2     ./run_wfc.py --model_name_or_path bert-base-uncased --do_train --do_eval --output_dir out_fever_dbg     --top_k 1 --logging_steps 50 --num_train_epochs 3 --warmup_steps 50 --gradient_accumulation_steps 16     --learning_rate 5e-05  --overwrite_output_dir --maxnum 30000           

    conda activate wfc;     cd ~/wfc-hons;     CUDA_VISIBLE_DEVICES=2,0,1     ./run_wfc.py --model_name_or_path bert-base-uncased --do_train --do_eval --output_dir out_fever_dbg     --top_k 1 --logging_steps 50 --num_train_epochs 3 --warmup_steps 50 --gradient_accumulation_steps 16     --learning_rate 5e-05 --levenshtein_only --overwrite_output_dir --maxnum 30000           

    conda activate wfc;     cd ~/wfc-hons;     CUDA_VISIBLE_DEVICES=1,3,0     ./run_wfc.py --model_name_or_path bert-base-uncased --do_train --do_eval --output_dir out_fever_dbg     --top_k 3 --logging_steps 50 --num_train_epochs 3 --warmup_steps 50 --gradient_a

In [5]:
len(cmds)

6

In [6]:
# cmds = [' conda activate wfc;     cd ~/wfc-hons;     CUDA_VISIBLE_DEVICES=0,1,2     ./run_wfc.py --model_name_or_path bert-base-uncased --do_train --do_eval --output_dir out_apr28     --top_k 5 --logging_steps 50 --num_train_epochs 2 --warmup_steps 50 --gradient_accumulation_steps 4     --learning_rate 5e-05  --overwrite_output_dir --maxnum 10000           ']

In [7]:
# for i in range(0, len(cmds), 4):
processes = [Popen(cmd , shell=True) for cmd in cmds[:]]
    # do other things here..
    # wait for completion
    # for p in processes: p.wait()

In [ ]:
for p in processes: p.kill(9)